In [7]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [74]:
data = pd.read_csv('data/combinedClimateFiresDataset_2010.csv')

### Remove extra columns from data and create labels

In [75]:
# data = data.drop(columns=['Unnamed: 0.1','Unnamed: 0_x', 'Unnamed: 0'])
# labels = pd.DataFrame()
# labels['FIRE_SIZE'] = data['FIRE_SIZE']
# labels['FIRE_SIZE_CLASS'] = data['FIRE_SIZE_CLASS']
# data = data.drop(columns=['FIRE_SIZE_CLASS','FIRE_SIZE'])

data.head()

,Unnamed: 0,Unnamed: 0_x,Unnamed: 0.1,OBJECTID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,SOURCE_REPORTING_UNIT,FIRE_NAME,FIRE_YEAR,DATE_x,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,LATITUDE_x,LONGITUDE_x,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_NAME,Shape,merging,Unnamed: 0_y,STATION,STATION_NAME,ELEVATION,LATITUDE_y,LONGITUDE_y,DATE_y,DLY-GRDD-BASE40,DLY-GRDD-BASE45,DLY-GRDD-BASE50,DLY-GRDD-BASE55,DLY-GRDD-BASE57,DLY-GRDD-BASE60,DLY-GRDD-BASE65,DLY-GRDD-BASE70,DLY-GRDD-BASE72,DLY-GRDD-TB4886,DLY-GRDD-TB5086,DLY-CLDD-BASE45,DLY-CLDD-BASE50,DLY-CLDD-BASE55,DLY-CLDD-BASE57,DLY-CLDD-BASE60,DLY-CLDD-NORMAL,DLY-CLDD-BASE70,DLY-CLDD-BASE72,DLY-HTDD-BASE40,DLY-HTDD-BASE45,DLY-HTDD-BASE50,DLY-HTDD-BASE55,DLY-HTDD-BASE57,DLY-HTDD-BASE60,DLY-HTDD-NORMAL,DLY-PRCP-25PCTL,DLY-SNWD-25PCTL,DLY-SNOW-25PCTL,DLY-PRCP-50PCTL,DLY-SNWD-50PCTL,DLY-SNOW-50PCTL,DLY-PRCP-75PCTL,DLY-SNWD-75PCTL,DLY-SNOW-75PCTL,MTD-PRCP-NORMAL,MTD-SNOW-NORMAL,YTD-PRCP-NORMAL,YTD-SNOW-NORMAL,DLY-PRCP-PCTALL-GE001HI,DLY-PRCP-PCTALL-GE010HI,DLY-PRCP-PCTALL-GE050HI,DLY-PRCP-PCTALL-GE100HI,DLY-SNWD-PCTALL-GE001WI,DLY-SNWD-PCTALL-GE010WI,DLY-SNWD-PCTALL-GE003WI,DLY-SNWD-PCTALL-GE005WI,DLY-SNOW-PCTALL-GE001TI,DLY-SNOW-PCTALL-GE010TI,DLY-SNOW-PCTALL-GE100TI,DLY-SNOW-PCTALL-GE030TI,DLY-SNOW-PCTALL-GE050TI,DLY-TAVG-NORMAL,DLY-DUTR-NORMAL,DLY-TMAX-NORMAL,DLY-TMIN-NORMAL,DLY-TAVG-STDDEV,DLY-DUTR-STDDEV,DLY-TMAX-STDDEV,DLY-TMIN-STDDEV,longLatOLS
0,0,66041,1067487,1067488,NONFED,ST-CACDF,ST/C&L,USCABEU,CABEU,25,2010,2010-05-19,139,NaN,2.0,Equipment Use,NaN,NaN,NaN,36.767,-121.303,14.0,MISSING/NOT SPECIFIED,CA,NaN,NaN,b'\x00\x01\xad\x10\x00\x00\xc0\x07\x1bCeS^\xc0...,0,70948,GHCND:USC00045933,MOUNT HAMILTON CA US,1282.0,37.34361,-121.64250,2010-05-19,16,11,7,4,3,2,1,-7777,-7777,10,9,11,7,4,3,2,1,-7777,-7777,-7777,1,2,3,5,6,10,0.04,-9999,-66.6,0.12,-9999,-66.6,0.25,-9999,-66.6,0.78,0.0,15.95,14.3,155,89,15,4,0,0,0,0,1,0,0,0,0,55.9,15.1,63.4,48.3,9.7,5.0,9.9,10.3,NaN
1,1,66042,1067488,1067489,NONFED,ST-CACDF,ST/C&L,USCABEU,CABEU,BOLADO,2010,2010-06-08,159,NaN,9.0,Miscellaneous,NaN,NaN,NaN,36.777,-121.311,14.0,MISSING/NOT SPECIFIED,CA,NaN,NaN,b'\x00\x01\xad\x10\x00\x00\xd0\xb7\x92>\xe9S^\...,1,80093,GHCND:USW00093243,MERCED 23 WSW CA US,23.8,37.23810,-120.88250,2010-06-08,28,23,18,13,11,8,4,1,-7777,19,18,23,18,13,11,8,4,1,-7777,0,0,0,0,-7777,-7777,1,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,0.06,-9999.0,7.31,-9999.0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,68.0,39.6,87.8,48.2,-9999.0,-9999.0,-9999.0,-9999.0,NaN
2,2,66043,1067489,1067490,NONFED,ST-CACDF,ST/C&L,USCABEU,CABEU,SANTA ANA RD HOLLISTER,2010,2010-06-12,163,NaN,9.0,Miscellaneous,NaN,NaN,NaN,36.856,-121.381,14.0,MISSING/NOT SPECIFIED,CA,NaN,NaN,b'\x00\x01\xad\x10\x00\x00\xe4e\xda\x1fdX^\xc0...,2,68417,GHCND:USW00093228,HAYWARD AIR TERMINAL CA US,13.1,37.65420,-122.11500,2010-06-12,23,18,13,8,6,4,1,-7777,-7777,15,13,18,13,8,6,4,1,-7777,-7777,0,0,0,0,-7777,-7777,3,-6.66,-9999,-9999.0,-6.66,-9999,-9999.0,-6.66,-9999,-9999.0,0.12,-9999.0,11.31,-9999.0,31,13,0,0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,63.2,16.8,71.6,54.8,4.6,6.7,7.6,2.8,NaN
3,3,66044,1067490,1067491,NONFED,ST-CACDF,ST/C&L,USCABEU,CABEU,HOSPITAL,2010,2010-06-14,165,NaN,9.0,Miscellaneous,NaN,NaN,NaN,36.818,-121.391,14.0,MISSING/NOT SPECIFIED,CA,NaN,NaN,b'\x00\x01\xad\x10\x00\x00T\xa3\xe4\xf6\x07Y^\...,3,141054,GHCND:USC00048999,TRACY CARBONA CA US,41.1,37.68194,-121.34667,2010-06-14,31,26,21,16,14,11,6,3,2,21,20,26,21,16,14,11,6,3,2,0,0,0,-7777,-7777,-7777,-7777,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,-9999.00,-9999,-9999.0,0.10,-9999.0,8.52,-9999.0,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,71.2,32.7,87.5,54.8,-9999.0,-9999.0,-9999.0,-9999.0,NaN
4,4,66045,1067491,1067492,NONFED,ST-CACDF,ST/C&L,USCABEU,CABEU,HWY 101 S/ HWY 129,2010,2010-07-11,192,NaN,9.0,Miscellaneous,NaN

In [104]:
# data = data.drop(columns=['DATE_x','DATE_y', 'DISCOVERY_TIME', 'CONT_DATE',
#                           'CONT_TIME','SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
#                           'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
#                           'SOURCE_REPORTING_UNIT', 'FIRE_NAME', 'OWNER_DESCR',
#                           'Shape', 'STATION', 'STATION_NAME', 'STAT_CAUSE_DESCR'])
# data = data.drop(columns=['STATE', 'COUNTY', 'FIPS_NAME'])

features = data[['DLY-GRDD-BASE40', 'DLY-GRDD-BASE45', 'DLY-GRDD-BASE50',
       'DLY-GRDD-BASE55', 'DLY-GRDD-BASE57', 'DLY-GRDD-BASE60',
       'DLY-GRDD-BASE65', 'DLY-GRDD-BASE70', 'DLY-GRDD-BASE72',
       'DLY-GRDD-TB4886', 'DLY-GRDD-TB5086', 'DLY-CLDD-BASE45',
       'DLY-CLDD-BASE50', 'DLY-CLDD-BASE55', 'DLY-CLDD-BASE57',
       'DLY-CLDD-BASE60', 'DLY-CLDD-NORMAL', 'DLY-CLDD-BASE70',
       'DLY-CLDD-BASE72', 'DLY-HTDD-BASE40', 'DLY-HTDD-BASE45',
       'DLY-HTDD-BASE50', 'DLY-HTDD-BASE55', 'DLY-HTDD-BASE57',
       'DLY-HTDD-BASE60', 'DLY-HTDD-NORMAL', 'DLY-PRCP-25PCTL',
       'DLY-SNWD-25PCTL', 'DLY-SNOW-25PCTL', 'DLY-PRCP-50PCTL',
       'DLY-SNWD-50PCTL', 'DLY-SNOW-50PCTL', 'DLY-PRCP-75PCTL',
       'DLY-SNWD-75PCTL', 'DLY-SNOW-75PCTL', 'MTD-PRCP-NORMAL',
       'MTD-SNOW-NORMAL', 'YTD-PRCP-NORMAL', 'YTD-SNOW-NORMAL',
       'DLY-PRCP-PCTALL-GE001HI', 'DLY-PRCP-PCTALL-GE010HI',
       'DLY-PRCP-PCTALL-GE050HI', 'DLY-PRCP-PCTALL-GE100HI',
       'DLY-SNWD-PCTALL-GE001WI', 'DLY-SNWD-PCTALL-GE010WI',
       'DLY-SNWD-PCTALL-GE003WI', 'DLY-SNWD-PCTALL-GE005WI',
       'DLY-SNOW-PCTALL-GE001TI', 'DLY-SNOW-PCTALL-GE010TI',
       'DLY-SNOW-PCTALL-GE100TI', 'DLY-SNOW-PCTALL-GE030TI',
       'DLY-SNOW-PCTALL-GE050TI', 'DLY-TAVG-NORMAL', 'DLY-DUTR-NORMAL',
       'DLY-TMAX-NORMAL', 'DLY-TMIN-NORMAL', 'DLY-TAVG-STDDEV',
       'DLY-DUTR-STDDEV', 'DLY-TMAX-STDDEV', 'DLY-TMIN-STDDEV',]]

In [105]:
features.columns.values
# data.dtypes


array(['DLY-GRDD-BASE40', 'DLY-GRDD-BASE45', 'DLY-GRDD-BASE50',
       'DLY-GRDD-BASE55', 'DLY-GRDD-BASE57', 'DLY-GRDD-BASE60',
       'DLY-GRDD-BASE65', 'DLY-GRDD-BASE70', 'DLY-GRDD-BASE72',
       'DLY-GRDD-TB4886', 'DLY-GRDD-TB5086', 'DLY-CLDD-BASE45',
       'DLY-CLDD-BASE50', 'DLY-CLDD-BASE55', 'DLY-CLDD-BASE57',
       'DLY-CLDD-BASE60', 'DLY-CLDD-NORMAL', 'DLY-CLDD-BASE70',
       'DLY-CLDD-BASE72', 'DLY-HTDD-BASE40', 'DLY-HTDD-BASE45',
       'DLY-HTDD-BASE50', 'DLY-HTDD-BASE55', 'DLY-HTDD-BASE57',
       'DLY-HTDD-BASE60', 'DLY-HTDD-NORMAL', 'DLY-PRCP-25PCTL',
       'DLY-SNWD-25PCTL', 'DLY-SNOW-25PCTL', 'DLY-PRCP-50PCTL',
       'DLY-SNWD-50PCTL', 'DLY-SNOW-50PCTL', 'DLY-PRCP-75PCTL',
       'DLY-SNWD-75PCTL', 'DLY-SNOW-75PCTL', 'MTD-PRCP-NORMAL',
       'MTD-SNOW-NORMAL', 'YTD-PRCP-NORMAL', 'YTD-SNOW-NORMAL',
       'DLY-PRCP-PCTALL-GE001HI', 'DLY-PRCP-PCTALL-GE010HI',
       'DLY-PRCP-PCTALL-GE050HI', 'DLY-PRCP-PCTALL-GE100HI',
       'DLY-SNWD-PCTALL-GE001WI', 'DLY-SNWD-PC

In [106]:
# Replace negative values by na

features[features < 0] = None
# features[features < 0] = None
features

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:3163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)


,DLY-GRDD-BASE40,DLY-GRDD-BASE45,DLY-GRDD-BASE50,DLY-GRDD-BASE55,DLY-GRDD-BASE57,DLY-GRDD-BASE60,DLY-GRDD-BASE65,DLY-GRDD-BASE70,DLY-GRDD-BASE72,DLY-GRDD-TB4886,DLY-GRDD-TB5086,DLY-CLDD-BASE45,DLY-CLDD-BASE50,DLY-CLDD-BASE55,DLY-CLDD-BASE57,DLY-CLDD-BASE60,DLY-CLDD-NORMAL,DLY-CLDD-BASE70,DLY-CLDD-BASE72,DLY-HTDD-BASE40,DLY-HTDD-BASE45,DLY-HTDD-BASE50,DLY-HTDD-BASE55,DLY-HTDD-BASE57,DLY-HTDD-BASE60,DLY-HTDD-NORMAL,DLY-PRCP-25PCTL,DLY-SNWD-25PCTL,DLY-SNOW-25PCTL,DLY-PRCP-50PCTL,DLY-SNWD-50PCTL,DLY-SNOW-50PCTL,DLY-PRCP-75PCTL,DLY-SNWD-75PCTL,DLY-SNOW-75PCTL,MTD-PRCP-NORMAL,MTD-SNOW-NORMAL,YTD-PRCP-NORMAL,YTD-SNOW-NORMAL,DLY-PRCP-PCTALL-GE001HI,DLY-PRCP-PCTALL-GE010HI,DLY-PRCP-PCTALL-GE050HI,DLY-PRCP-PCTALL-GE100HI,DLY-SNWD-PCTALL-GE001WI,DLY-SNWD-PCTALL-GE010WI,DLY-SNWD-PCTALL-GE003WI,DLY-SNWD-PCTALL-GE005WI,DLY-SNOW-PCTALL-GE001TI,DLY-SNOW-PCTALL-GE010TI,DLY-SNOW-PCTALL-GE100TI,DLY-SNOW-PCTALL-GE030TI,DLY-SNOW-PCTALL-GE050TI,DLY-TAVG-NORMAL,DLY-DUTR-NORMAL,DLY-TMAX-NORMAL,DLY-TMIN-NORMAL,DLY-TAVG-STDDEV,DLY-DUTR-STDDEV,DLY-TMAX-STDDEV,DLY-TMIN-STDDEV
0,16.0,11.0,7.0,4.0,3.0,2.0,1.0,NaN,NaN,10.0,9.0,11.0,7.0,4.0,3.0,2.0,1.0,NaN,NaN,NaN,1.0,2.0,3.0,5.0,6.0,10.0,0.04,NaN,NaN,0.12,NaN,NaN,0.25,NaN,NaN,0.78,0.0,15.95,14.3,155.0,89.0,15.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,55.9,15.1,63.4,48.3,9.7,5.0,9.9,10.3
1,28.0,23.0,18.0,13.0,11.0,8.0,4.0,1.0,NaN,19.0,18.0,23.0,18.0,13.0,11.0,8.0,4.0,1.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.06,NaN,7.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,39.6,87.8,48.2,NaN,NaN,NaN,NaN
2,23.0,18.0,13.0,8.0,6.0,4.0,1.0,NaN,NaN,15.0,13.0,18.0,13.0,8.0,6.0,4.0,1.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.12,NaN,11.31,NaN,31.0,13.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.2,16.8,71.6,54.8,4.6,6.7,7.6,2.8
3,31.0,26.0,21.0,16.0,14.0,11.0,6.0,3.0,2.0,21.0,20.0,26.0,21.0,16.0,14.0,11.0,6.0,3.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,NaN,8.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.2,32.7,87.5,54.8,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01,0.0,5.81,0.0,7.0,2.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,40.0,34.0,29.0,24.0,22.0,19.0,14.0,10.0,8.0,26.0,24.0,34.0,29.0,24.0,22.0,19.0,14.0,10.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0.0,6.57,0.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.5,32.9,96.0,63.0,5.3,4.2,6.0,5.4
6,31.0,26.0,21.0,16.0,14.0,11.0,6.0,2.0,2.0,21.0,19.0,26.0,21.0,16.0,14.0,11.0,6.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.04,0.0,9.78,0.0,34.0,16.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.0,31.0,86.5,55.5,5.0,7.9,8.1,4.0
7,16.0,11.0,6.0,2.0,1.0,1.0,NaN,NaN,0.0,9.0,7.0,11.0,6.0,2.0,1.0,1.0,NaN,NaN,0.0,0.0,NaN,NaN,1.0,2.0,4.0,9.0,0.06,NaN,NaN,0.21,NaN,NaN,0.51,NaN,NaN,2.04,0.0,11.25,0.0,331.0,223.0,84.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.2,15.3,63.9,48.5,4.4,5.8,6.3,4.0
8,16.0,11.0,7.0,3.0,2.0,1.0,NaN,NaN,NaN,13.0,12.0,11.0,7.0,3.0,2.0,1.0,NaN,NaN,NaN,0.0,NaN,NaN,2.0,3.0,5.0,9.0,0.07,NaN,NaN,0.18,NaN,NaN,0.36,NaN,NaN,0.03,0.0,13.87,0.2,112.0,77.0,18.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.1,37.1,74.7,37.5,6.0,10.3,9.5,5.9
9,24.0,19.0,14.0,9.0,7.0,4.0,1.0,NaN,NaN,16.0,14.0,19.0,14.0,9.0,7.0,4.0,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.06,NaN,12.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.8,23.6,75.6,52.0,3.4,6.4,5.4,3.9


In [107]:
missing_percentage = []
for column in features.columns:
    missing_percentage.append(str(int(features[column].isna().sum()/len(features[column])*100)) + "% missing in "+ column)
missing_percentage.sort()
missing_percentage

['18% missing in DLY-CLDD-BASE45',
 '18% missing in DLY-CLDD-BASE50',
 '18% missing in DLY-DUTR-NORMAL',
 '18% missing in DLY-GRDD-BASE40',
 '18% missing in DLY-GRDD-BASE45',
 '18% missing in DLY-GRDD-BASE50',
 '18% missing in DLY-GRDD-TB4886',
 '18% missing in DLY-GRDD-TB5086',
 '18% missing in DLY-TAVG-NORMAL',
 '18% missing in DLY-TMAX-NORMAL',
 '18% missing in DLY-TMIN-NORMAL',
 '2% missing in MTD-PRCP-NORMAL',
 '2% missing in YTD-PRCP-NORMAL',
 '20% missing in DLY-CLDD-BASE55',
 '20% missing in DLY-GRDD-BASE55',
 '21% missing in DLY-CLDD-BASE57',
 '21% missing in DLY-GRDD-BASE57',
 '22% missing in DLY-PRCP-PCTALL-GE001HI',
 '24% missing in DLY-CLDD-BASE60',
 '24% missing in DLY-GRDD-BASE60',
 '26% missing in DLY-DUTR-STDDEV',
 '26% missing in DLY-HTDD-BASE40',
 '26% missing in DLY-PRCP-PCTALL-GE010HI',
 '26% missing in DLY-TAVG-STDDEV',
 '26% missing in DLY-TMAX-STDDEV',
 '26% missing in DLY-TMIN-STDDEV',
 '28% missing in DLY-PRCP-PCTALL-GE050HI',
 '29% missing in DLY-CLDD-NORMAL'

## Predicting FIRE_SIZE_CLASS
### TO-DO
* Create dummy variables for categorical data

In [97]:
from sklearn.feature_selection import SelectKBest, chi2, RFE, VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from itertools import compress


In [102]:
new_features = SelectKBest(chi2, k=10).fit(features, labels.FIRE_SIZE_CLASS)
model = RandomForestClassifier()
rfe = RFE(model, 7)

In [92]:
rfe.fit(features, labels.FIRE_SIZE_CLASS)

RFE(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
  n_features_to_select=7, step=1, verbose=0)

In [103]:
list(compress(features.columns.values, new_features.get_support()))

['YTD-SNOW-NORMAL',
 'DLY-PRCP-PCTALL-GE001HI',
 'DLY-PRCP-PCTALL-GE010HI',
 'DLY-PRCP-PCTALL-GE050HI',
 'DLY-SNWD-PCTALL-GE001WI',
 'DLY-SNWD-PCTALL-GE010WI',
 'DLY-SNWD-PCTALL-GE003WI',
 'DLY-SNWD-PCTALL-GE005WI',
 'DLY-SNOW-PCTALL-GE001TI',
 'DLY-SNOW-PCTALL-GE010TI']

In [100]:
list(compress(features.columns.values, rfe.get_support()))

['MTD-PRCP-NORMAL',
 'YTD-PRCP-NORMAL',
 'DLY-PRCP-PCTALL-GE001HI',
 'DLY-TAVG-NORMAL',
 'DLY-DUTR-NORMAL',
 'DLY-TMAX-NORMAL',
 'DLY-TMIN-NORMAL']